In [2]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# One hot encoding.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [5]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

# Without Data Augmentation

In [6]:
def generate_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
  ])

model = generate_model()

In [12]:
batch_size = 64
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=40, batch_size=batch_size)


# validate the model on test dataset to determine generalization
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))

Epoch 1/40
782/782 [==============================] - 11s 13ms/step - loss: 0.6466 - accuracy: 0.7795
Epoch 2/40
782/782 [==============================] - 10s 13ms/step - loss: 0.6430 - accuracy: 0.7805
Epoch 3/40
782/782 [==============================] - 10s 13ms/step - loss: 0.6406 - accuracy: 0.7834
Epoch 4/40
782/782 [==============================] - 10s 13ms/step - loss: 0.6304 - accuracy: 0.7841
Epoch 5/40
782/782 [==============================] - 11s 14ms/step - loss: 0.6250 - accuracy: 0.7879
Epoch 6/40
782/782 [==============================] - 10s 13ms/step - loss: 0.6210 - accuracy: 0.7892
Epoch 7/40
782/782 [==============================] - 11s 13ms/step - loss: 0.6185 - accuracy: 0.7898
Epoch 8/40
782/782 [==============================] - 10s 13ms/step - loss: 0.6163 - accuracy: 0.7932
Epoch 9/40
782/782 [==============================] - 10s 13ms/step - loss: 0.6135 - accuracy: 0.7919
Epoch 10/40
782/782 [==============================] - 10s 13ms/step - loss: 0.611

In [13]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.97):
            print("\nReached 97% accuracy so cancelling training!")
            self.model.stop_training = True 

# Data Augmenteation

In [14]:
datagen = ImageDataGenerator(
            zca_epsilon=1e-06,
            rotation_range=10,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.1,
            fill_mode='nearest',
            cval=0.,
            horizontal_flip=True
            )

In [15]:
datagen.fit(x_train)

In [16]:
callbacks = myCallback()

history = model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=64),
                                    epochs=40,
                                    validation_data=(x_test, y_test),
                                    callbacks=[callbacks],
                                    workers=4)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/40
782/782 [==============================] - 48s 61ms/step - loss: 0.8528 - accuracy: 0.7138 - val_loss: 0.7882 - val_accuracy: 0.7399
Epoch 2/40
782/782 [==============================] - 49s 63ms/step - loss: 0.8269 - accuracy: 0.7220 - val_loss: 0.7167 - val_accuracy: 0.7611
Epoch 3/40
782/782 [==============================] - 48s 62ms/step - loss: 0.8332 - accuracy: 0.7224 - val_loss: 0.7456 - val_accuracy: 0.7535
Epoch 4/40
782/782 [==============================] - 49s 62ms/step - loss: 0.8232 - accuracy: 0.7235 - val_loss: 0.7863 - val_accuracy: 0.7366
Epoch 5/40
782/782 [==============================] - 48s 62ms/step - loss: 0.8213 - accuracy: 0.7230 - val_loss: 0.7295 - val_accuracy: 0.7517
Epoch 6/40
782/782 [==============================] - 48s 62ms/step - loss: 0.8068 - accuracy: 0.7286 - val_loss: 0.7492 - val_accuracy: 0.7486
Epoch 7/40
782/782 [==============================] - 48s 62